# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), May 6, 2020**

Let's get started by loading the `DataFrames` package.

In [1]:
using DataFrames, Random

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [2]:
DataFrame()

""
""


Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [3]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]))

,A,B,C
,Int64,Float64,String
1,1,0.050034,e5U
2,2,0.746714,OuX
3,3,0.754752,HXE


We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [4]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])
DataFrame(x)

,A,B,C
,Int64,Bool,Char
1,1,1,'a'
2,2,0,'b'


Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use `Symbol`s to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [5]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
,Int64,Bool,Char
1,1,1,'a'
2,2,0,'b'


Although, in general, using `Symbol`s rather than strings to denote column names is preferred (as it is faster) DataFrames.jl accepts passing strings as column names, so this also works:

In [6]:
DataFrame("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])

,A,B,C
,Int64,Bool,Char
1,1,1,'a'
2,2,0,'b'


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [7]:
DataFrame([rand(3) for i in 1:3])

,x1,x2,x3
,Float64,Float64,Float64
1,0.150228,0.596849,0.0807212
2,0.752575,0.803518,0.420067
3,0.0497649,0.749983,0.696673


It is not allowed to pass a vector of scalars to `DataFrame` constructor.

In [8]:
DataFrame(rand(3))

ArgumentError: ArgumentError: 'Array{Float64,1}' iterates 'Float64' values, which doesn't satisfy the Tables.jl `AbstractRow` interface

Instead use a transposed vector if you have a vector of single values (in this way you effectively pass a two dimensional array to the constructor which is supported).

In [9]:
DataFrame(permutedims([1, 2, 3]))

,x1,x2,x3
,Int64,Int64,Int64
1,1,2,3


Note, however, that in other cases you can pass a single value to a `DataFrame` constrtuctor and it will be broadcasted appropriately

In [10]:
DataFrame("A" => 1, "B" => [1,2,3])

,A,B
,Int64,Int64
1,1,1
2,1,2
3,1,3


In [11]:
DataFrame(A=1, B=[1,2,3], C=Ref(["a", "b"]))

,A,B,C
,Int64,Int64,Array…
1,1,1,"[""a"", ""b""]"
2,1,2,"[""a"", ""b""]"
3,1,3,"[""a"", ""b""]"


As you can see, as a particular rule, like in standard broadcasting - values wrapped in `Ref` get unwrapped as can be seen in the example above. Now column `:C` contains three times the same vector `["a", "b"]` (the values are aliases, not copies).

You can also pass a vector of `NamedTuple`s to construct a `DataFrame`:

In [12]:
v = [(a=1, b=2), (a=3, b=4)]
DataFrame(v)

,a,b
,Int64,Int64
1,1,2
2,3,4


If you pass a vector of vectors then pass a second argument to give the columns names.

In [13]:
DataFrame([1:3, 4:6, 7:9], [:A, :B, :C])

,A,B,C
,Int64,Int64,Int64
1,1,4,7
2,2,5,8
3,3,6,9


or

In [14]:
DataFrame([1:3, 4:6, 7:9], ["A", "B", "C"])

,A,B,C
,Int64,Int64,Int64
1,1,4,7
2,2,5,8
3,3,6,9


Alternatively you can pass a `NamedTuple` of vectors:

In [15]:
n = (a=1:3, b=11:13)
DataFrame(n)

,a,b
,Int64,Int64
1,1,11
2,2,12
3,3,13


Here we create a `DataFrame` from a matrix,

In [16]:
DataFrame(rand(3,4))

,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,0.214943,0.627109,0.106318,0.545103
2,0.889415,0.704955,0.0711917,0.123521
3,0.910963,0.519658,0.345197,0.628368


and here we do the same but also pass column names.

In [17]:
DataFrame(rand(3,4), Symbol.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.805002,0.321617,0.973277,0.607407
2,0.657194,0.591434,0.88867,0.811593
3,0.023967,0.123394,0.450165,0.237314


or

In [18]:
DataFrame(rand(3,4), string.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.425596,0.669391,0.015079,0.353956
2,0.507025,0.751827,0.88226,0.120275
3,0.0190897,0.524012,0.217928,0.517043


We can also construct an uninitialized DataFrame.

Here we pass column types, names and number of rows to get an unitialized `DataFrame`; we get `missing` in column :D because `Any >: Missing`.

(in all examples below we omit the version with column names passed as strings, but it is allowed everywhere where `Symbol`s are accepted)

In [19]:
DataFrame([Int, Float64, String, Any], [:A, :B, :C, :D], 1)

,A,B,C,D
,Int64,Float64,String,Any
1,35,3.9e-322,#undef,missing


To initialize a `DataFrame` with column names, but no rows use

In [20]:
DataFrame([Int, Float64, String], [:A, :B, :C])

,A,B,C
,Int64,Float64,String


or

In [21]:
DataFrame(A=Int[], B=Float64[], C=String[])

,A,B,C
,Int64,Float64,String


Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` also copies the vectors.

In [22]:
x = DataFrame(a=1:2, b='a':'b')
y = copy(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, false)

Calling `DataFrame` on a `DataFrame` object works like `copy`.

In [23]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, false)

You can avoid copying of columns of a data frame (if it is possible) by passing `copycols=false` keyword argument or using `DataFrame!` constructor.

In [24]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x, copycols=false)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, true)

In [25]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame!(x)
(x === y), isequal(x, y), (x.a == y.a), (x.a === y.a)

(false, true, true, true)

The same rule applies to other constructors

In [26]:
a = [1, 2, 3]
df1 = DataFrame(a=a)
df2 = DataFrame(a=a, copycols=false)
df1.a === a, df2.a === a

(false, true)

You can create a similar uninitialized `DataFrame` based on an original one:

In [27]:
similar(x)

,a,b
,Int64,Char
1,139778165174576,'\xa3\xe1\xf2\xc0'
2,139777388798976,'\x00\x00\x7f\x20'


number of rows in a new DataFrame can be passed as a second argument

In [28]:
similar(x, 0)

,a,b
,Int64,Char


You can also create a new `DataFrame` from `SubDataFrame` or `DataFrameRow` (discussed in detail later in the tutorial; in particular although `DataFrameRow` is considered a 1-dimensional object similar to a `NamedTuple` it gets converted to a 1-row `DataFrame` for convinience)

In [29]:
sdf = view(x, [1,1], :)

,a,b
,Int64,Char
1,1,'a'
2,1,'a'


In [30]:
typeof(sdf)

SubDataFrame{DataFrame,DataFrames.Index,Array{Int64,1}}

In [31]:
DataFrame(sdf)

,a,b
,Int64,Char
1,1,'a'
2,1,'a'


In [32]:
dfr = x[1, :]

,a,b
,Int64,Char
1,1,'a'


In [33]:
DataFrame(dfr)

,a,b
,Int64,Char
1,1,'a'


### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [34]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix` or `Array`.

In [35]:
Matrix(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

In [36]:
Array(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [37]:
x = DataFrame(x=1:2, y=[missing,"B"])

,x,y
,Int64,String?
1,1,missing
2,2,B


In [38]:
Matrix(x)

2×2 Array{Any,2}:
 1  missing
 2  "B"

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [39]:
x = DataFrame(x=1:2, y=3:4)

,x,y
,Int64,Int64
1,1,3
2,2,4


In [40]:
Matrix(x)

2×2 Array{Int64,2}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [41]:
x = DataFrame(x=1:2, y=[missing,4])

,x,y
,Int64,Int64?
1,1,missing
2,2,4


In [42]:
Matrix(x)

2×2 Array{Union{Missing, Int64},2}:
 1   missing
 2  4

Note that we can't force a conversion of `missing` values to `Int`s!

In [43]:
Matrix{Int}(x)

ArgumentError: ArgumentError: cannot convert a DataFrame containing missing values to Matrix{Int64} (found for column y)

### Conversion to `NamedTuple` related tabular structures

First define some data frame

In [44]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


Now we convert a `DataFrame` into a `NamedTuple` of vectors

In [45]:
ct = Tables.columntable(x)

(x = [1, 2], y = ["A", "B"])

Next we convert it into a vector of `NamedTuples`

In [46]:
rt = Tables.rowtable(x)

2-element Array{NamedTuple{(:x, :y),Tuple{Int64,String}},1}:
 (x = 1, y = "A")
 (x = 2, y = "B")

We can perform the conversions back to a `DataFrame` using a standard constructor call:

In [47]:
DataFrame(ct)

,x,y
,Int64,String
1,1,A
2,2,B


In [48]:
DataFrame(rt)

,x,y
,Int64,String
1,1,A
2,2,B


### Iterating data frame by rows or columns

Sometimes it is useful to create a wrapper around a `DataFrame` that produces its rows or columns.

For iterating columns you can use the `eachcol` function.

In [49]:
ec = eachcol(x)

,x,y
,Int64,String
1,1,A
2,2,B


`DataFrameColumns` object behaves as a vector (note though that is is a view - it does not create)

In [50]:
ec isa AbstractVector

true

In [51]:
ec[1]

2-element Array{Int64,1}:
 1
 2

similarly `eachrow` creates a `DataFrameRows` object that is a vector of its rows

In [52]:
er = eachrow(x)

,x,y
,Int64,String
1,1,A
2,2,B


In [53]:
er isa AbstractVector

true

In [54]:
er[end]

,x,y
,Int64,String
2,2,B


Note that both data frame and also `DataFrameColumns` and `DataFrameRows` objects are not type stable (they do not know the types of their columns). This is useful to avoid compilation cost if you have very wide data frames with heterogenous column types.

However, often (especially if a data frame is narrows) it is useful to create a lazy iterator that produces `NamedTuple`s for each row of the `DataFrame`. Its key benefit is that it is type stable (so it is useful when you want to perform some operations in a fast way on a small subset of columns of a `DataFrame` - this strategy is often used internally by DataFrames.jl package):

In [55]:
nti = Tables.namedtupleiterator(x)

Tables.NamedTupleIterator{Tables.Schema{(:x, :y),Tuple{Int64,String}},Tables.RowIterator{NamedTuple{(:x, :y),Tuple{Array{Int64,1},Array{String,1}}}}}(Tables.RowIterator{NamedTuple{(:x, :y),Tuple{Array{Int64,1},Array{String,1}}}}((x = [1, 2], y = ["A", "B"]), 2))

In [56]:
for row in enumerate(nti)
    @show row
end

row = (1, (x = 1, y = "A"))
row = (2, (x = 2, y = "B"))


similarly to the previous options you can easily convert `NamedTupleIterator` back to a `DataFrame`.

In [57]:
DataFrame(nti)

,x,y
,Int64,String
1,1,A
2,2,B


### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [58]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

,a,a_2,a_1
,Int64,Int64,Int64
1,1,2,3


Otherwise, duplicates are not allowed.

In [59]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

ArgumentError: ArgumentError: Duplicate variable names: :a. Pass makeunique=true to make them unique using a suffix automatically.

Finallly observe that `nothing` is not printed when displaying a `DataFrame`:

In [60]:
DataFrame(x=[1, nothing], y=[nothing, "a"], z=[missing, "c"])

,x,y,z
,Union…,Union…,String?
1,1,,missing
2,,a,c
